## 논리 게이트 구현1

In [1]:
# AND 게이트 구현
def AND_gate(x1, x2):
    w1 = 1
    w2 = 1
    b = -1
    result = x1 * w1 + x2 * w2 + b
    if result <= 0:
        return 0
    else:
        return 1

In [2]:
# OR 게이트 구현 예
def OR_gate(x1, x2):
    w1 = 1
    w2 = 1
    b = -0.5
    result = x1 * w1 + x2 * w2 + b
    if result <= 0:
        return 0
    else:
        return 1

In [3]:
# NAND 게이트 구현 예
def NAND_gate(x1, x2):
    w1 = -0.5
    w2 = -0.5
    b = 1
    result = x1 * w1 + x2 * w2 + b
    if result <= 0:
        return 0
    else:
        return 1

In [4]:
# XOR 게이트 구현 예
def XOR_gate(x1, x2):
    z1 = NAND_gate(x1, x2)
    z2 = OR_gate(x1, x2)
    y = AND_gate(z1, z2)
    
    return y

In [5]:
print("AND gate")
for xs in [(0, 0), (0, 1), (1, 0), (1, 1)]:
    y = AND_gate(xs[0], xs[1])
    print(str(xs) + " -> " + str(y))

AND gate
(0, 0) -> 0
(0, 1) -> 0
(1, 0) -> 0
(1, 1) -> 1


In [6]:
print("NAND gate")
for xs in [(0, 0), (0, 1), (1, 0), (1, 1)]:
    y = NAND_gate(xs[0], xs[1])
    print(str(xs) + " -> " + str(y))

NAND gate
(0, 0) -> 1
(0, 1) -> 1
(1, 0) -> 1
(1, 1) -> 0


In [7]:
print("OR gate")
for xs in [(0, 0), (0, 1), (1, 0), (1, 1)]:
    y = OR_gate(xs[0], xs[1])
    print(str(xs) + " -> " + str(y))

OR gate
(0, 0) -> 0
(0, 1) -> 1
(1, 0) -> 1
(1, 1) -> 1


In [8]:
print("XOR_gate")
for xs in [(0, 0), (0, 1), (1, 0), (1, 1)]:
    y = XOR_gate(xs[0], xs[1])
    print(str(xs) + " -> " + str(y))

XOR_gate
(0, 0) -> 0
(0, 1) -> 1
(1, 0) -> 1
(1, 1) -> 0


## 논리 게이트 구현 2

In [9]:
# 도구 임포트
import torch
import torch.nn as nn

# GPU 연산이 가능할 경우 GPU 연산 수행
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [10]:
# 각 gate 문제에 해당하는 입력과 출력 정의
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
# Y = torch.FloatTensor([[0], [0], [0], [1]]).to(device) # AND
# Y = torch.FloatTensor([[1], [1], [1], [0]]).to(device) # NAND
# Y = torch.FloatTensor([[0], [1], [1], [1]]).to(device) # OR
Y = torch.FloatTensor([[0], [1], [1], [1]]).to(device) # XOR

In [14]:
# 다층 퍼셉트론 설계
linear1 = nn.Linear(2, 2, bias=True)
sigmoid1 = nn.Sigmoid()
linear2 = nn.Linear(2, 1, bias=True)
sigmoid2 = nn.Sigmoid()
model = nn.Sequential(linear1, sigmoid1, linear2, sigmoid2).to(device)

In [15]:
# 이진 분류 문제이므로 비용 함수로 크로스엔트로피 함수 사용
# nn.BCELoss() - 이진 분류에서 사용하는 크로스엔트로피 함수
# 비용 함수와 옵티마이저 정의
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [16]:
# 반복 학습 수행
for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0: # 100번째 에포크마다 비용 출력
        print(step, cost.item())

print("\n\n")
print("hypothesis 값: ")
print(hypothesis)

print("\n")
predicted = (hypothesis > 0.5).float()
print("예측 값: ")
print(predicted)

print("\n")
print("실제 값: ")
print(Y)

0 0.5895378589630127
100 0.4898300766944885
200 0.06494033336639404
300 0.02224024385213852
400 0.012598431669175625
500 0.008618536405265331
600 0.0064917439594864845
700 0.0051815323531627655
800 0.004298396408557892
900 0.003665090072900057
1000 0.0031898804008960724
1100 0.0028208326548337936
1200 0.0025262637063860893
1300 0.0022859242744743824
1400 0.002086350694298744
1500 0.0019180189119651914
1600 0.0017741984920576215
1700 0.001649974612519145
1800 0.0015416033565998077
1900 0.001446329290047288
2000 0.0013618930242955685
2100 0.0012865436729043722
2200 0.0012189347762614489
2300 0.0011579446727409959
2400 0.0011026313295587897
2500 0.0010522620286792517
2600 0.0010062090586870909
2700 0.0009639493655413389
2800 0.0009250046568922698
2900 0.0008890014141798019
3000 0.0008556705433875322
3100 0.0008247581426985562
3200 0.0007959058275446296
3300 0.0007689790800213814
3400 0.0007437688764184713
3500 0.0007201555999927223
3600 0.0006979601457715034
3700 0.0006770481122657657
380